In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
import time
import os.path as op

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm_notebook

from kiki import KikiNet
from data_torch import MaskedUntouched2DDataset
from torch_training import fit_torch

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

n_samples_train = 7135
n_samples_val = 7135

n_volumes_train = 199
n_volumes_val = 199

In [4]:
# data loader
# generators
AF = 4
train_gen = MaskedUntouched2DDataset(train_path, af=AF, inner_slices=1)
val_gen = MaskedUntouched2DDataset(val_path, af=AF)
train_loader = DataLoader(
    dataset=train_gen,
    batch_size=1,
    shuffle=False,
    num_workers=35,
#     pin_memory=True,
)
val_loader = DataLoader(
    dataset=val_gen,
    batch_size=1,
    num_workers=35,
#     pin_memory=True,
)

In [5]:
run_params = {
    'n_cascade': 2,
    'n_convs': 25,
    'n_filters': 64,
}
n_epochs = 100
run_id = f'kikinet_torch_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
log_dir = op.join('logs', run_id)
print(run_id)

cascadenet_torch_af4_1567094646


In [6]:
model = KikiNet(**run_params)
optimizer = Adam(model.parameters(), lr=1e-3)
writer = SummaryWriter(log_dir=log_dir)

In [7]:
%%time
fit_torch(
    model, 
    train_loader, 
    val_loader=None, 
    epochs=1,
    hard_limit_train=0,
    hard_limit_val=0,
    writer=None,
    optimizer=optimizer,
    chkpt_path=chkpt_path,
    run_id=run_id,
    device='cpu',
    save_freq=100,
    tqdm_wrapper=tqdm_notebook,
)

CPU times: user 6min 20s, sys: 1min 15s, total: 7min 35s
Wall time: 25.9 s
